# ORE Version 12 - New Analytics

This dashboard demonstrates the following new analytics added in ORE version 12:
- Scenario
- Historical Simulation Var
- PNL
- PNL Explain

## Set up and load utility functions
To install ORE run **pip install open-source-risk-engine**

Dependencies:
 * pandas, numpy, lxml
 * ORE environmental variable

In [6]:
import ORE as ore

# utility script available
import sys
sys.path.append('..')
import utilities
import numpy as np
import os
import pandas as pd

from lxml import etree
def decodeXML(filename):
    return etree.tostring(etree.parse(filename)).decode('UTF-8')


## Scenario

 - Example 57
 - Returns the base scenario from a Simulation Market for a given date. 
 - The scenario is a representation of the state of the market.
 - It is a set of key/value pairs, keyed on RiskFactor:
     *  DiscountCurve/EUR/0: 1.00013288
     *  DiscountCurve/EUR/1: 1.00028576
     *  FXSpot/USDEUR/0: 0.8929
 - Requires a Simulation Parameters configuration which specifies the risk factors.
 - Scenario generation needed for Historical Simulation VaR

**ORE XML for Scenario Analytic**

In [ ]:
oreXml = decodeXML("Input/Example_57/ore.xml")
print(oreXml)

\
**Simulation XML for Scenario Analytic**

In [ ]:
simMarketParamsXml = decodeXML("Input/Example_57/simulation.xml")
print(simMarketParamsXml)

\
**Setup the Scenario Analytic**

In [ ]:
inputs = ore.InputParameters()

# set asof date
inputs.setAsOfDate("2016-02-05")
inputs.setResultsPath(".")
inputs.setAllFixings(True)
inputs.setEntireMarket(True)

# Read in the XML configurations
curveConfigsXml = decodeXML("../../Input/curveconfig.xml")
conventionsXml = decodeXML("../../Input/conventions.xml")
pricingEngineXml = decodeXML("../../Input/pricingengine.xml")
todaysMarketXml = decodeXML("../../Input/todaysmarket.xml")

# set configurations in InputParameters
inputs.setCurveConfigs(curveConfigsXml)
inputs.setConventions(conventionsXml)
inputs.setPricingEngine(pricingEngineXml)
inputs.setTodaysMarketParams(todaysMarketXml)

with open("../../Input/market_20160205_flat.txt") as f:
    market_data = ore.StrVector(f.read().splitlines())
    
with open("../../Input/fixings_20160205.txt") as f:
    fixings_data = ore.StrVector(f.read().splitlines())

**Set Scenario specific parameters**

In [ ]:
inputs.insertAnalytic("SCENARIO")
inputs.setScenarioSimMarketParams(decodeXML("Input/Example_57/simulation.xml"))

**Run the Scenario Analytic**

In [ ]:
ore_app = ore.OREApp(inputs, "log.txt", 63)
ore_app.run(market_data, fixings_data)

In [ ]:
ore_app.getReportNames()

In [ ]:
r = ore_app.getReport('scenario')
utilities.format_report(r)


## Historical Simulation VaR

 - Example 58
 - Generates a historical simulation VaR calculation given a portfolio and a set of historical market scenarios covering
a historical observation period.
 - Load the raw historical market scenarios and use these to generate "shift" scenarios to reflect market moves for the historical period
 - Apply each shift scenario to todays simulated market sequentially, and revalue the portfolio to generate a set of PNLs
 - Calculate VaR at the quantiles provided


\
**Historical Simulation ORE XML**

In [7]:
oreXml = decodeXML("Input/Example_58/ore.xml")
print(oreXml)

<ORE>
  <Setup>
    <Parameter name="asofDate">2019-12-30</Parameter>
    <Parameter name="inputPath">Input</Parameter>
    <Parameter name="outputPath">Output</Parameter>
    <Parameter name="logFile">log.txt</Parameter>
    <Parameter name="logMask">31</Parameter>
    <Parameter name="marketDataFile">market.txt</Parameter>
    <Parameter name="fixingDataFile">fixings.txt</Parameter>
    <Parameter name="implyTodaysFixings">N</Parameter>
    <Parameter name="curveConfigFile">curveconfig.xml</Parameter>
    <Parameter name="conventionsFile">conventions.xml</Parameter>
    <Parameter name="marketConfigFile">todaysmarket.xml</Parameter>
    <Parameter name="pricingEnginesFile">pricingengine.xml</Parameter>
    <Parameter name="observationModel">None</Parameter>
    <Parameter name="baseCurrency">EUR</Parameter>
    <Parameter name="portfolioFile">portfolio.xml</Parameter>
    <Parameter name="nThreads">1</Parameter>
  </Setup>
  <Markets>
    <Parameter name="lgmcalibration">default</Par

\
**Historical Scenarios**

In [ ]:
pd.read_csv('Input/Example_58/scenarios.csv')

\
**Set up Historical Simulation VAR calculation**

In [ ]:
inputs = ore.InputParameters()

# set asof date
inputs.setAsOfDate("2019-12-30")
inputs.setResultsPath(".")
inputs.setAllFixings(True)
inputs.setEntireMarket(True)
inputs.setBaseCurrency("EUR")

curveConfigsXml = decodeXML("Input/Example_58/curveconfig.xml")
conventionsXml = decodeXML("Input/Example_58/conventions.xml")
pricingEngineXml = decodeXML("Input/Example_58/pricingengine.xml")
todaysMarketXml = decodeXML("Input/Example_58/todaysmarket.xml")
portfolioXml = decodeXML("Input/Example_58/portfolio.xml")

# set configurations in InputParameters
inputs.setCurveConfigs(curveConfigsXml)
inputs.setConventions(conventionsXml)
inputs.setPricingEngine(pricingEngineXml)
inputs.setTodaysMarketParams(todaysMarketXml)
inputs.setPortfolio(portfolioXml)

with open("Input/Example_58/market.txt") as f:
    market_data = ore.StrVector(f.read().splitlines())
    
with open("Input/Example_58/fixings.txt") as f:
    fixings_data = ore.StrVector(f.read().splitlines())

\
**Historical Simulation VaR specific parameters**

In [ ]:
inputs.insertAnalytic("HISTSIM_VAR")
inputs.setScenarioReader("Input/Example_58/scenarios.csv")
inputs.setHistVarSimMarketParamsFromFile("Input/Example_58/simulation.xml")
inputs.setBenchmarkVarPeriod("2017-01-17,2019-12-30")
inputs.setMporDays(10)
inputs.setMporCalendar("USD")
inputs.setMporOverlappingPeriods(True)
inputs.setVarQuantiles("0.01,0.05,0.95,0.99")

**Run analytic**

In [ ]:
ore_app = ore.OREApp(inputs, "log.txt", 32)
ore_app.run(market_data, fixings_data)

In [ ]:
ore_app.getReportNames()

\
**VaR Report**

In [ ]:
r = ore_app.getReport('var')
utilities.format_report(r)




## PNL

- Example 62
- Calculate the PNL between 2 dates
- Generate a SimMarket and Scenario for each date, and use them to generate 4 flavours of NPV
    * NPV(t0)
    * NPV(asof=t0; mkt=t1) - Shift the t0 SimMarket to t1's market    * 
    * NPV(t1)
    * NPV(asof=t1; mkt=t0) - Shift the t1 SimMarket to t0's market
- Calculate PeriodCashFlow - Aggregate of trade flows in the period
- We use the NPV's to calculate:
    * Theta: NPV(asof=t1; mkt=t0) - NPV(t0) + PeriodCashFLow
    * HypotheticalCleanPnL: NPV(asof=t0; mkt=t1) - NPV(t0)
    * DirtyPnL: NPV(t1) - NPV(t0)
    * CleanPnL: NPV(t1) - NPV(t0) + PeriodCashFlow 

\
**PNL ORE XML**

In [8]:
oreXml = decodeXML("Input/Example_62/ore_pnl.xml")
print(oreXml)

<ORE>
  <Setup>
    <Parameter name="asofDate">2023-01-31</Parameter>
    <Parameter name="baseCurrency">USD</Parameter>
    <Parameter name="inputPath">Input</Parameter>
    <Parameter name="outputPath">Output/Pnl</Parameter>
    <Parameter name="logFile">log.txt</Parameter>
    <Parameter name="logMask">31</Parameter>
    <Parameter name="marketDataFile">market.txt</Parameter>
    <Parameter name="fixingDataFile">fixings.txt</Parameter>
    <Parameter name="implyTodaysFixings">Y</Parameter>
    <Parameter name="fixingCutoff">2023-02-14</Parameter>
    <Parameter name="curveConfigFile">curveconfig.xml</Parameter>
    <Parameter name="conventionsFile">conventions.xml</Parameter>
    <Parameter name="marketConfigFile">todaysmarket.xml</Parameter>
    <Parameter name="pricingEnginesFile">pricingengine.xml</Parameter>
    <Parameter name="portfolioFile">portfolio.xml</Parameter>
    <Parameter name="observationModel">None</Parameter>
    <Parameter name="continueOnError">false</Parameter>

\
**Set up PNL Analytic**

In [9]:
inputs = ore.InputParameters()

# set asof date
inputs.setAsOfDate("2023-01-31")
inputs.setResultsPath(".")
inputs.setAllFixings(True)
inputs.setEntireMarket(True)
inputs.setBaseCurrency("USD")

curveConfigsXml = decodeXML("Input/Example_62/curveconfig.xml")
conventionsXml = decodeXML("Input/Example_62/conventions.xml")
pricingEngineXml = decodeXML("Input/Example_62/pricingengine.xml")
todaysMarketXml = decodeXML("Input/Example_62/todaysmarket.xml")
portfolioXml = decodeXML("Input/Example_62/portfolio.xml")

# set configurations in InputParameters
inputs.setCurveConfigs(curveConfigsXml)
inputs.setConventions(conventionsXml)
inputs.setPricingEngine(pricingEngineXml)
inputs.setTodaysMarketParams(todaysMarketXml)
inputs.setPortfolio(portfolioXml)

with open("Input/Example_62/market.txt") as f:
    market_data = ore.StrVector(f.read().splitlines())
    
with open("Input/Example_62/fixings.txt") as f:
    fixings_data = ore.StrVector(f.read().splitlines())

\
**Set PNL specific parameters**

In [10]:
inputs.insertAnalytic("PNL")
inputs.setScenarioSimMarketParamsFromFile("Input/Example_62/simulation.xml")
inputs.setMporDays(10)
inputs.setMporCalendar("USD")

\
**Run PNL Analytic**

In [11]:
ore_app = ore.OREApp(inputs, "log.txt", 125, True)
ore_app.run(market_data, fixings_data)

In [12]:
ore_app.getReportNames()

('dividends',
 'fixings',
 'marketdata',
 'pnl',
 'pnl_cashflow',
 'pnl_npv_lagged_t0',
 'pnl_npv_lagged_t1',
 'pnl_npv_t0',
 'pnl_npv_t1',
 'pnl_npv_t1_port_t0',
 'pricingstats',
 'runtimes',
 'todaysmarketcalibration',
 'zero_scenarios')

In [13]:
r = ore_app.getReport('pnl')
utilities.format_report(r)

,TradeId,TradeType,Maturity,MaturityTime,StartDate,EndDate,NPV(t0),NPV(asof=t0;mkt=t1),NPV(asof=t1;mkt=t0),NPV(t1;portfolio=t0),...,PeriodCashFlow,New,Matured,Terminated,Amendments,Theta,HypotheticalCleanPnL,CleanPnL,DirtyPnL,Currency
0,SwapLeg,Swap,2024-02-01,1.002508,2023-01-31,2023-02-14,47640.849246,47618.392561,47725.150117,47703.486315,...,0.000000,0.0,0.0,0.0,0.0,84.300871,-22.456685,62.637069,62.637069,USD
1,SwapLegFlow,Swap,2024-02-05,1.013437,2023-01-31,2023-02-14,97311.809777,97288.862765,47568.552161,47546.718334,...,49861.111111,0.0,0.0,0.0,0.0,117.853495,-22.947012,96.019668,-49765.091443,USD



## PNL Explain

 - Example 62 
 - Extends the PNL analytic to "explain" the PNL in terms of IR/FX/EQ etc delta/gamma/vega buckets.
 - Run a sensitivity analysis to generate sensitivities for each trade
 - Bucket each risk factor based on risk type and risk class
 - RiskTypes:

<img src="risktype.jpg" width=500 height=500 />

 - RiskClasses:
   
<img src="riskclass.jpg" width=700 height=600 />

 - Create a set of filters containing risk type / risk class pairs, from every combination.
 - Consecutively loop through each risk filter and calculate the PNL for each bucket by multiplying the sensitivities by the scenario moves for all risk factors within that filter


**PNL Explain ORE XML**

In [14]:
oreXml = decodeXML("Input/Example_62/ore_explain.xml")
print(oreXml)

<ORE>
  <Setup>
    <Parameter name="asofDate">2023-01-31</Parameter>
    <Parameter name="baseCurrency">USD</Parameter>
    <Parameter name="inputPath">Input</Parameter>
    <Parameter name="outputPath">Output/PnlExplain</Parameter>
    <Parameter name="logFile">log.txt</Parameter>
    <Parameter name="logMask">31</Parameter>
    <Parameter name="marketDataFile">market.txt</Parameter>
    <Parameter name="fixingDataFile">fixings.txt</Parameter>
    <Parameter name="implyTodaysFixings">Y</Parameter>
    <Parameter name="fixingCutoff">2023-02-14</Parameter>
    <Parameter name="curveConfigFile">curveconfig.xml</Parameter>
    <Parameter name="conventionsFile">conventions.xml</Parameter>
    <Parameter name="marketConfigFile">todaysmarket.xml</Parameter>
    <Parameter name="pricingEnginesFile">pricingengine.xml</Parameter>
    <Parameter name="portfolioFile">portfolio.xml</Parameter>
    <Parameter name="observationModel">None</Parameter>
    <Parameter name="continueOnError">false</Par

\
**Sensitivity Config**

In [15]:
sensitivityXml = decodeXML("Input/Example_62/sensitivity.xml")
print(sensitivityXml)

<SensitivityAnalysis>
	<DiscountCurves>
		<DiscountCurve ccy="EUR">
			<ShiftType>Absolute</ShiftType>
			<ShiftSize>0.0001</ShiftSize>
			<ShiftScheme>Forward</ShiftScheme>
			<ShiftTenors>2W, 1M, 3M, 6M, 1Y, 2Y, 3Y, 5Y, 10Y, 15Y, 20Y, 30Y</ShiftTenors>
			<ParConversion>
				<Instruments>DEP, DEP, DEP, DEP, OIS, OIS, OIS, OIS, OIS, OIS, OIS, OIS</Instruments>
				<SingleCurve>true</SingleCurve>
				<Conventions>
					<Convention id="DEP">EUR-ON-DEPOSIT</Convention>
					<Convention id="OIS">EUR-OIS</Convention>
				</Conventions>
			</ParConversion>
		</DiscountCurve>
		<DiscountCurve ccy="USD">
			<ShiftType>Absolute</ShiftType>
			<ShiftSize>0.0001</ShiftSize>
			<ShiftScheme>Forward</ShiftScheme>
			<ShiftTenors>2W, 1M, 3M, 6M, 1Y, 2Y, 3Y, 5Y, 10Y, 15Y, 20Y, 30Y</ShiftTenors>
			<ParConversion>
				<Instruments>DEP, DEP, DEP, DEP, OIS, OIS, OIS, OIS, OIS, OIS, OIS, OIS</Instruments>
				<SingleCurve>true</SingleCurve>
				<Conventions>
					<Convention id="DEP">USD-ON-DEPOSIT</Con

\
**Set up the PNL Explain calculation**

In [16]:
inputs = ore.InputParameters()

# set asof date
inputs.setAsOfDate("2023-01-31")
inputs.setResultsPath(".")
inputs.setAllFixings(True)
inputs.setEntireMarket(True)
inputs.setBaseCurrency("USD")

curveConfigsXml = decodeXML("Input/Example_62/curveconfig.xml")
conventionsXml = decodeXML("Input/Example_62/conventions.xml")
pricingEngineXml = decodeXML("Input/Example_62/pricingengine.xml")
todaysMarketXml = decodeXML("Input/Example_62/todaysmarket.xml")
portfolioXml = decodeXML("Input/Example_62/portfolio.xml")

# set configurations in InputParameters
inputs.setCurveConfigs(curveConfigsXml)
inputs.setConventions(conventionsXml)
inputs.setPricingEngine(pricingEngineXml)
inputs.setTodaysMarketParams(todaysMarketXml)
inputs.setPortfolio(portfolioXml)

with open("Input/Example_62/market.txt") as f:
    market_data = ore.StrVector(f.read().splitlines())
    
with open("Input/Example_62/fixings.txt") as f:
    fixings_data = ore.StrVector(f.read().splitlines())

\
**Set PNL Explain specific parameters**

In [17]:
inputs.insertAnalytic("PNL_EXPLAIN")
inputs.setScenarioSimMarketParamsFromFile("Input/Example_62/simulation.xml")
inputs.setMporDate(ore.Date(14,2,2023))
# for sensitivity analysis
simulationXml = decodeXML("Input/Example_62/simulation.xml")
inputs.setSensiSimMarketParams(simulationXml)
inputs.setSensiScenarioData(sensitivityXml)

\
**Run PNL Explain Analytic**

In [19]:
ore_app = ore.OREApp(inputs, "log.txt", 125, True)
ore_app.run(market_data, fixings_data)

In [20]:
ore_app.getReportNames()

('dividends',
 'fixings',
 'marketdata',
 'pnl',
 'pnl_cashflow',
 'pnl_explain',
 'pnl_npv_lagged_t0',
 'pnl_npv_lagged_t1',
 'pnl_npv_t0',
 'pnl_npv_t1',
 'pnl_npv_t1_port_t0',
 'pricingstats',
 'runtimes',
 'sensitivity',
 'todaysmarketcalibration',
 'zero_scenarios')

In [21]:
r = ore_app.getReport('pnl_explain')
utilities.format_report(r)

,TradeId,TradeType,Maturity,MaturityTime,StartDate,EndDate,NPV(t0),NPV(asof=t0;mkt=t1),NPV(asof=t1;mkt=t0),NPV(t1;portfolio=t0),...,FxVega,InfDelta,InfGamma,InfVega,CreditDelta,CreditGamma,CreditVega,CommDelta,CommGamma,CommVega
0,SwapLeg,Swap,2024-02-01,1.002508,2023-01-31,2023-02-14,47640.849246,47618.392561,47725.150117,47703.486315,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,SwapLegFlow,Swap,2024-02-05,1.013437,2023-01-31,2023-02-14,97311.809777,97288.862765,47568.552161,47546.718334,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
utilities.writeReport(r, [0,12,16,17,18,19,20,21])


TradeId            New                Theta              HypotheticalCleanPnL  CleanPnL           DirtyPnL           Currency           ScenarioPnl        
SwapLeg            0.0000             84.3009            -22.4567           62.6371            62.6371            USD                -22.4556           
SwapLegFlow        0.0000             117.8535           -22.9470           96.0197            -49765.0914        USD                -22.9462           




## Verify PNL Explain Delta/Gamma Calculation

**Sensitivity and Scenario Output**

In [23]:
sensitivity = ore_app.getReport('sensitivity')
sensis = utilities.format_report(sensitivity)
sensis['Factor_1'] = sensis['Factor_1'].str[:-3]
sensis

,TradeId,IsPar,Factor_1,ShiftSize_1,Factor_2,ShiftSize_2,Currency,Base NPV,Delta,Gamma
0,SwapLeg,false,DiscountCurve/USD/4,0.0001,,0.0,USD,47640.849246,-4.750831,4.738087e-04
1,SwapLeg,false,DiscountCurve/USD/5,0.0001,,0.0,USD,47640.849246,-0.026069,1.426088e-08
2,SwapLegFlow,false,DiscountCurve/USD/0,0.0001,,0.0,USD,97311.809777,-0.081902,1.346052e-07
3,SwapLegFlow,false,DiscountCurve/USD/4,0.0001,,0.0,USD,97311.809777,-4.683722,4.619964e-04
4,SwapLegFlow,false,DiscountCurve/USD/5,0.0001,,0.0,USD,97311.809777,-0.129927,3.554451e-07


In [31]:
scenarios = ore_app.getReport('zero_scenarios')
utilities.writeReport(scenarios, [0,19,20])


Date               DiscountCurve/USD/4  DiscountCurve/USD/5  
2023-01-31         0.9529             0.9187             
2023-02-14         0.9525             0.9180             


In [32]:
t = utilities.format_report(scenarios)[['DiscountCurve/USD/4','DiscountCurve/USD/5']].T
t.columns = ['Scenario_t0','Scenario_t1']
df = sensis[sensis['TradeId']=='SwapLeg'][['TradeId','Factor_1','ShiftSize_1','Delta','Gamma']]
df['Time'] = df['Factor_1'].map({'DiscountCurve/USD/4': 1, 'DiscountCurve/USD/5': 2})
df = df.join(t, on='Factor_1', how='left')
df['ScenarioShift'] = np.log(df['Scenario_t1'] / df['Scenario_t0']) / (df['Time'] * df['ShiftSize_1'])
df['ExplainedDelta'] = df['Delta'] * df['ScenarioShift']
df['ExplainedGamma'] = 0.5 * df['Gamma'] * df['ScenarioShift'] * df['ScenarioShift']
df

,TradeId,Factor_1,ShiftSize_1,Delta,Gamma,Time,Scenario_t0,Scenario_t1,ScenarioShift,ExplainedDelta,ExplainedGamma
0,SwapLeg,DiscountCurve/USD/4,0.0001,-4.750831,4.738087e-04,1,0.952938,0.952489,-4.705121,22.353239,5.244628e-03
1,SwapLeg,DiscountCurve/USD/5,0.0001,-0.026069,1.426088e-08,2,0.918710,0.917950,-4.134170,0.107773,1.218689e-07


In [33]:
df.groupby(['TradeId'])[['ExplainedDelta', 'ExplainedGamma']].sum()

,ExplainedDelta,ExplainedGamma
TradeId,,
SwapLeg,22.461012,0.005245
